## Project setup, imports, etc

In [2]:
# approach 2 - PRAW
import requests
import praw
import pandas as pd
import hashlib
from datetime import datetime
from dateutil import tz
from collections import defaultdict

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
CLIENT_ID = 'Mx4J1DKZ856aT1ug9glOdA'
SECRET_TOKEN = 'j0a57wDUiC9TSDQhx1CYNz4jl65Aiw'
USERNAME = 'malconst'
PASSWORD = '7(>vAn!GqC1"'
USER_AGENT = 'Seeker/0.0.1'

In [3]:
# Establish authorized Reddit instance
reddit = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = SECRET_TOKEN,
    user_agent = USER_AGENT,
    username = USERNAME,
    password = PASSWORD
)

## Search Reddit
PRAW Search doucmentation https://praw.readthedocs.io/en/latest/code_overview/models/subreddit.html?highlight=search#praw.models.Subreddit.search

*Important:* the search function in this implementation is a method on the Subreddit object, not Subreddits

For now, implemented hashing as described here https://www.codegrepper.com/code-examples/python/python+convert+string+to+hash

In [122]:
#Example of a stored dictionary, to test comparing hashes and accepting/rejecting the search result
stored_search_test = {
    'searches': [{
            'query': 'olivia',
            'search_results': [{
                'result_id': 'u0wzyl',
                'body': {
                    'author': 'shirouu-kun',
                    'title': '[US-CA] [H] Built Think6.5 v2 Pink Assassin, Think6.5 v2 Solder PCB, 2U Badges, Alpaca and Silent Alpaca Switches, JTK Night Sakura, Latrialum [W] Paypal',
                    'created date': '2022-04-10 20:13:00',
                    'url': 'https://www.reddit.com/r/mechmarket/comments/u0wzyl/usca_h_built_think65_v2_pink_assassin_think65_v2/',
                    'hash': '9278f1a8736b8f34f1cd26a36994a22cd527809a1bdd367f34482f69f32e4143'
                }
            }]
        }, {
            'query': '9009',
            'search_results': [{
                'result_id': 'u08jd9',
                'body': {
                    'author': 'TradersParad1se',
                    'title': '[US-TX] [H] Keycult 2/65 Ocean Grey/ Silver, Tofu65 Grey Aluminum, Bauer 2 PC/Cyan, GMK Modo Light, GMK, GMK Modo2 [W] Paypal, GMK Redacted, GMK Dracula + Nightmode, GMK 9009',
                    'created date': '2022-04-09 20:52:43',
                    'url': 'https://www.reddit.com/r/mechmarket/comments/u08jd9/ustx_h_keycult_265_ocean_grey_silver_tofu65_grey/',
                    'hash': 'f673a1d4ea5b049c407723788a3086235a7e5ebf3ca4d10fac0fb16e53d05312'
                }
            }]
        }, {
            'query': 'hammerhead',
            'search_results': [{
                'result_id': 'u0oded',
                'body': {
                    'author': 'WestC0ast_BestC0ast',
                    'title': '[US-CA] [H] Below Cost Keyboards and GMK Keycaps [W] PayPal',
                    'created date': '2022-04-10 12:59:16',
                    'url': 'https://www.reddit.com/r/mechmarket/comments/u0oded/usca_h_below_cost_keyboards_and_gmk_keycaps_w/',
                    'hash': 'c46e932a561f99d41129267fcd5ba7f5534a61846f7265dd967106ee44e721a3'
                }
            }]
        },
        [{
            'query': 'olivia',
            'search_results': [{
                'result_id': 'u0yci3',
                'body': {
                    'author': Redditor(name = 'danteThrives'),
                    'title': '[US-AZ] [H] Paypal or Local Cash [W] Zoom65',
                    'created date': '2022-04-10 21:27:34',
                    'url': 'https://www.reddit.com/r/mechmarket/comments/u0yci3/usaz_h_paypal_or_local_cash_w_zoom65/',
                    'hash': 'f4029b1d8b501516b8f2db16f07fc237f788585ff5c99b2d53756dc52737580c'
                }
            }]
        }]
    ]
}

In [151]:
subreddit = reddit.subreddit("mechmarket")

# get dictionary instead of make it
search_dict = {}

searches = [] #holder for final storage
queries = ["olivia","9009","hammerhead"]
limit = 1


for query in queries:
    results = [] #holder for handling each set of search results
    for result in subreddit.search(query,sort="new", limit=limit): #https://praw.readthedocs.io/en/latest/code_overview/models/subreddit.html?highlight=search#praw.models.Subreddit.search
        # Structure the data before storing - there ahs to be a better way than this... like convering to JSON upstream
        result_id = str(result)
        to_hash = str(result.author)+ str(result.title)+ str(result.url)
        hash_object = hashlib.sha256(to_hash.encode('utf-8'))
        hex_dig = hash_object.hexdigest()

        #compare hash to existing dictionary

        #if store = true
        body = {
            'author':result.author,
            'title':result.title,
            'created date': datetime.fromtimestamp(result.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
            'url':result.url,
            'hash':hex_dig
        }
        results.append({'result_id': result_id, 'body': body}) #just bundle the result_id and body into its own result object and add it to the the results list 
        
    searches.append({'query':query,'search_results':results})
search_dict['searches'] = searches #add the final list

# store dictionary somewhere


In [ ]:
# Copy of above cell to work out 
#   pulling from a database (in this case stored_search_test)
#   matching new search result hex against database hex
#   storing if no match, discarding if match
#   updating the search_dict (for current-search results) as usual
#   updating the database

# could definitely abstract some of the loops to be their own functions

## Also, realized you can just run the cell above and work with a memory copy of "search_dict" so... did that. Then it "stopped working", but since it's just two searches back to back when comparing the hashes with search_dict (because search_dict is from the cell above), so it wouldn't store anything since it all matches
## So this wasn't a useful refactor, but does validate that the code works by testing the other case

subreddit = reddit.subreddit("mechmarket")

# get dictionary instead of make it
search_dict = {}

searches = [] #holder for final storage
queries = ["olivia","9009","hammerhead"]
limit = 3


for query in queries:
    results = [] #holder for handling each set of search results
    for result in subreddit.search(query,sort="new", limit=limit): #https://praw.readthedocs.io/en/latest/code_overview/models/subreddit.html?highlight=search#praw.models.Subreddit.search
        # Structure the data before storing - there ahs to be a better way than this... like convering to JSON upstream
        result_id = str(result)
        to_hash = str(result.author)+ str(result.title)+ str(result.url)
        hash_object = hashlib.sha256(to_hash.encode('utf-8'))
        hex_dig = hash_object.hexdigest()

        #Evaluate the new hex against the hexes in the database search results
        # build a list of all the hashes in the dict, then see if it's in there, if it is. don't store it
        hash_list = []
        for i in range(len(stored_search_test['searches'])):
            for j in range(len(stored_search_test['searches'][i]['search_results'])):
                hash_list.append(stored_search_test['searches'][i]['search_results'][j]['body']['hash'])
        if hex_dig in hash_list:
            print('breaking')
            break
        else:
            print('adding ', hex_dig)
            body = {
                'author':result.author,
                'title':result.title,
                'created date': datetime.fromtimestamp(result.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'url':result.url,
                'hash':hex_dig
            }
            results.append({'result_id': result_id, 'body': body}) #just bundle the result_id and body into its own result object and add it to the the results list
        searches.append({'query':query,'search_results':results})
search_dict['searches'].append(searches) #add the final list
stored_search_test['searches'].append(searches) # update the stored_dictionary

# store dictionary somewhere

In [ ]:
print('search_dict after second search; results - ', stored_search_test)

## Graveyard

In [55]:
# #Approach 1 - 
# import requests
# import praw

# # note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
# CLIENT_ID = 'Mx4J1DKZ856aT1ug9glOdA'
# SECRET_TOKEN = 'j0a57wDUiC9TSDQhx1CYNz4jl65Aiw'
# USERNAME = 'malconst'
# PASSWORD = '7(>vAn!GqC1"'
# USER_AGENT = 'Seeker/0.0.1'

# auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_TOKEN)

# # here we pass our login method (password), username, and password
# data = {'grant_type': 'password',
#         'username': USERNAME,
#         'password': PASSWORD}

# # setup our header info, which gives reddit a brief description of our app
# headers = {'User-Agent': 'Seeker/0.0.1'}

# # send our request for an OAuth token
# res = requests.post('https://www.reddit.com/api/v1/access_token',
#                     auth=auth, data=data, headers=headers)

# # convert response to JSON and pull access_token value
# TOKEN = res.json()

# # add authorization to our headers dictionary
# headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# res = requests.get("https://oauth.reddit.com/r/python/hot",
#                    headers=headers).json()

# print(res.url)  # let's see what we get

